Monte Carlo simulations of [ballistic precision estimates](http://ballistipedia.com/index.php?title=Closed_Form_Precision) from sample targets

In [3]:
"""
Statistical estimates and inference of Rayleigh distribution in ballistic target data
"""
import numpy as np
import math
from scipy.stats.distributions import chi2

In [133]:
class group:
    """Estimators of Rayleigh sigma, center, and confidence from shot groups"""
    cG = {}  # Memoize cG correction factor
    cR = {}  # center correction factor

    def __init__(self, n: int, shots: list[list[float]]=[], sigma: float=1.0, runOrderStatistics: bool=False):
        """
        :param n: Number of shots
        :param shots: List of (x,y) coordinates of center of each shot
            If no data provided then all simulation samples will be drawn from X,Y~N(0,1)
            in which case we know the true parameter we're estimating is sigma=1 with center at the origin
        """
        g = 1
        self.n = n
        self.degrees = 2*(n-g)  # Degrees of freedom of sigma estimate as chi^2-variate
        self.sigma = sigma
        if n not in group.cG:
            d = self.degrees + 1
            group.cG[n] = 1/math.exp(math.log(math.sqrt(2/(d-1)))+math.lgamma(d/2)-math.lgamma((d-1)/2))
            group.cR[n] = math.exp(math.log(math.sqrt(d/math.pi)) + d * math.log(4) + math.lgamma(d+1) + math.lgamma(d) - math.lgamma(2*d+1))

        if len(shots) > 0:
            self.shots = shots
        else:  # If we didn't receive a group of shots then create one from standard normal
            self.shots = [np.random.normal(scale=sigma, size=2) for i in range(0,n)]

        self.sampleCenter = np.mean(self.shots, axis=0)
        self.centeredShots = np.add(self.shots, -self.sampleCenter) # Shots adjusted so their centroid = (0,0)
        self.radii = np.linalg.norm(self.centeredShots, axis=-1)    # List of radii from sample center
        self.sumR2 = np.dot(self.radii, self.radii)
        self.sigmaEstimate = group.cG[n] * math.sqrt(self.sumR2 / self.degrees)  # Unbiased Rayleigh parameter estimate
        self.sigmaUpperConfidence = math.sqrt(self.sumR2 / chi2.ppf(0.05, self.degrees))  # Upper bound of 90% confidence interval on estimate

        # Sample distance to center
        self.d2c = math.sqrt(np.dot(self.sampleCenter, self.sampleCenter))
        # Median expected distance from sample center to true center = cR * sqrt(2*math.log(2)) * sigmaEstimate / sqrt(n)
        self.d2cMedian = group.cR[n] * 1.177410022515474 * self.sigmaEstimate / math.sqrt(n)

        if runOrderStatistics: self.sigmaFromOrderStatistics()

    def sigmaFromOrderStatistics(self):
        # Ref http://ballistipedia.com/index.php?title=File:Order_Statistics_of_Exponential_Distribution_in_Censored_Samples.pdf
        def Rstat(m: int):
            """Return mth smallest radii -- i.e., order statistic R(m)"""
            if m > self.n:
                raise Exception(f"Tried to access order statistic {m} on group size {self.n}")
            return self.radii[self.sortedShots][m-1]
        
        self.sortedShots = np.argsort(self.radii)  # Index of shot by radius, small to large
        if self.n == 3:
            self.orderSampleR2 = 0.66 * Rstat(3)**2
        elif self.n == 5:
            self.orderSampleR2 = 0.7792 * Rstat(4)**2
        elif self.n == 10:
            self.orderSampleR2 = 0.4913 * Rstat(6)**2 + 0.3030 * Rstat(9)**2
        else:
            raise Exception(f"No order statistics defined for n={self.n}")
        self.orderSampleSumR2 = self.n * self.orderSampleR2
        self.sigmaOrderStatEstimate = group.cG[self.n] * math.sqrt(self.orderSampleSumR2 / self.degrees)
        self.sigmaOrderStatUpperConfidence = math.sqrt(self.orderSampleSumR2 / (chi2.ppf(0.05, self.degrees)))
        return self.sigmaOrderStatEstimate


# Simulations
For validation.  Note that the Upper limit of the 90% confidence interval should be < true parameter 5% of the time.

### Sample center distance from true center

In [118]:
sim10 = [group(10) for i in range(0,10000)]
sigmaEstimates = [s.sigmaEstimate for s in sim10]
sigmaUppers = [s.sigmaUpperConfidence for s in sim10]
d2c = [s.d2c for s in sim10]
Ed2c = [s.d2cMedian for s in sim10]
print(f"Sigma   mean={np.mean(sigmaEstimates):.10f} \
      \t90%Conf<Sigma={sum(1 for x in sigmaUppers if x < sim10[0].sigma)/len(sigmaUppers):.2%} \
      \tMeanR ={np.mean(d2c):.3f} \
      \tR<Median={sum(1 for i in range(0, len(sigmaEstimates)) if d2c[i] > Ed2c[i])/len(sigmaEstimates):.2%}")


Sigma   mean=0.9981297366       	90%Conf<Sigma=5.18%       	MeanR =0.399       	R<Ed2c=50.33%


In [119]:
sim3 = [group(3) for i in range(0,10000)]
sigmaEstimates = [s.sigmaEstimate for s in sim3]
sigmaUppers = [s.sigmaUpperConfidence for s in sim3]
d2c = [s.d2c for s in sim3]
Ed2c = [s.d2cMedian for s in sim3]
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f} \
      \t90%Conf<Sigma={sum(1 for x in sigmaUppers if x < sim10[0].sigma)/len(sigmaUppers):.2%} \
      \tMeanR ={np.mean(d2c):.3f} \
      \tR<Ed2c={sum(1 for i in range(0, len(sigmaEstimates)) if d2c[i] > Ed2c[i])/len(sigmaEstimates):.2%}")


Sigma   mean=1.000       	90%Conf<Sigma=5.01%       	MeanR =0.722       	R<Ed2c=50.38%


### Order Statistics

In [136]:
# n = 10
sim = [group(10, runOrderStatistics=True) for i in range(0,10000)]
sigmaEstimates = [s.sigmaEstimate for s in sim]
sigmaUppers = [s.sigmaUpperConfidence for s in sim]
sigmaOSestimates = [s.sigmaOrderStatEstimate for s in sim]
sigmaOSuppers = [s.sigmaOrderStatUpperConfidence for s in sim]
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f}\tVar={np.var(sigmaEstimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaUppers if x < 1)/len(sigmaUppers):.2%}")
print(f"SigmaOS mean={np.mean(sigmaOSestimates):.3f}\tVar={np.var(sigmaOSestimates):.3f}\t90%OSConfidence<Parameter={sum(1 for x in sigmaOSuppers if x < 1)/len(sigmaOSuppers):.2%}")
print(f"\t\tOrder Stat Efficiency = {(np.var(sigmaEstimates)/np.var(sigmaOSestimates)):.3f}")

Sigma   mean=1.001	Var=0.028	90%Confidence<Parameter=4.89%
SigmaOS mean=1.001	Var=0.033	90%OSConfidence<Parameter=5.95%
		Order Stat Efficiency = 0.860


In [123]:
# Verify that variance matches the order-statistic variance when we use Efficiency*n = 86% * 10 ≈ 9 samples for the best sigma estimator
sim = [group(9) for i in range(0,10000)]
sigmaEstimates = [s.sigmaEstimate for s in sim]
sigmaUppers = [s.sigmaUpperConfidence for s in sim]
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f}\tVar={np.var(sigmaEstimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaUppers if x < 1)/len(sigmaUppers):.2%}")

Sigma   mean=0.999	median=0.993	Var=0.032	90%Confidence<Parameter=5.10%


In [138]:
# n = 5
sim = [group(5, runOrderStatistics=True) for i in range(0,10000)]
sigmaEstimates = [s.sigmaEstimate for s in sim]
sigmaUppers = [s.sigmaUpperConfidence for s in sim]
sigmaOSestimates = [s.sigmaOrderStatEstimate for s in sim]
sigmaOSuppers = [s.sigmaOrderStatUpperConfidence for s in sim]
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f}\tVar={np.var(sigmaEstimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaUppers if x < 1)/len(sigmaUppers):.2%}")
print(f"SigmaOS mean={np.mean(sigmaOSestimates):.3f}\tVar={np.var(sigmaOSestimates):.3f}\t90%OSConfidence<Parameter={sum(1 for x in sigmaOSuppers if x < 1)/len(sigmaOSuppers):.2%}")
print(f"\t\tOrder Stat Efficiency = {(np.var(sigmaEstimates)/np.var(sigmaOSestimates)):.3f}")

Sigma   mean=1.002	Var=0.064	90%Confidence<Parameter=4.79%
SigmaOS mean=1.014	Var=0.081	90%OSConfidence<Parameter=6.10%
		Order Stat Efficiency = 0.791


In [139]:
# n = 3 ... at this size a single order statistic is a very coarse estimator
sim = [group(3, runOrderStatistics=True) for i in range(0,50000)]
sigmaEstimates = [s.sigmaEstimate for s in sim]
sigmaUppers = [s.sigmaUpperConfidence for s in sim]
sigmaOSestimates = [s.sigmaOrderStatEstimate for s in sim]
sigmaOSuppers = [s.sigmaOrderStatUpperConfidence for s in sim]
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f}\tVar={np.var(sigmaEstimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaUppers if x < 1)/len(sigmaUppers):.2%}")
print(f"SigmaOS mean={np.mean(sigmaOSestimates):.3f}\tVar={np.var(sigmaOSestimates):.3f}\t90%OSConfidence<Parameter={sum(1 for x in sigmaOSuppers if x < 1)/len(sigmaOSuppers):.2%}")
print(f"\t\tOrder Stat Efficiency = {(np.var(sigmaEstimates)/np.var(sigmaOSestimates)):.3f}")

Sigma   mean=1.000	Var=0.132	90%Confidence<Parameter=5.06%
SigmaOS mean=1.041	Var=0.149	90%OSConfidence<Parameter=4.55%
		Order Stat Efficiency = 0.884
